In [1]:
import csv
import numpy as np
import pandas as pd

In [76]:
x = []
y = []
with open ('/content/data.csv') as file:
  csv_reader = csv.reader(file)
  for row in csv_reader:
    if row[0]=='ID':
      continue
    datum = []
    datum.append(row[3])
    datum.append(row[4])
    if row[1]=='Female':
      datum.append(0)
    else:
      datum.append(1)
    datum.append(float(row[2]))
    bmi = float(row[4])/(float(row[3])**2)
    datum.append(bmi)
    if row[5]=='yes':
      datum.append(0)
    else:
      datum.append(1)
    if row[6]=='yes':
      datum.append(0)
    else:
      datum.append(1)
    datum.append(float(row[7]))
    datum.append(float(row[8]))
    if row[9]=='no':
      datum.append(0)
    elif row[9]=='Sometimes':
      datum.append(1)
    elif row[9]=='Frequently':
      datum.append(2)
    else:
      datum.append(3)
    if row[10]=='yes':
      datum.append(0)
    else:
      datum.append(1)
    datum.append(float(row[11]))
    #if row[12]=='yes':
      #datum.append(0)
    #else:
      #datum.append(1)
    datum.append(float(row[13]))
    datum.append(float(row[14]))
    if row[15]=='no':
      datum.append(0)
    elif row[15]=='Sometimes':
      datum.append(1)
    elif row[15]=='Frequently':
      datum.append(2)
    else:
      datum.append(3)
    #if row[16]=='Walking':
      #datum.append(0)
    #elif row[16]=='Bike':
      #datum.append(1)
    #elif row[16]=='Public_Transportion':
      #datum.append(2)
    #elif row[16]=='Motorbike':
      #datum.append(3)
    #else:
      #datum.append(4)
    if row[17]=='Insufficient_Weight':
      y.append(0)
    elif row[17]=='Normal_Weight':
      y.append(1)
    elif row[17]=='Overweight_Level_I':
      y.append(2)
    elif row[17]=='Overweight_Level_II':
      y.append(3)
    elif row[17]=='Obesity_Type_I':
      y.append(4)
    elif row[17]=='Obesity_Type_II':
      y.append(5)
    else:
      y.append(6)
    x.append(datum)
print(x)
print(y)
x = np.array(x)
y = np.array(y)

[['1.624831', '69.975607', 0, 18.83919, 26.505150938561666, 0, 0, 2.253998, 2.752318, 1, 1, 2.328331, 0.815509, 0.024088, 0], ['1.755643', '137.796884', 0, 21.853826, 44.70611989614442, 0, 0, 3.0, 3.0, 1, 1, 2.184707, 1.978631, 0.838957, 1], ['1.806827', '108.829395', 1, 18.0, 33.33596806335821, 0, 0, 2.0, 1.24884, 1, 1, 2.416213, 1.0, 1.546738, 0], ['1.824901', '120.805715', 1, 24.149036, 36.27512033303055, 0, 0, 2.225149, 3.0, 1, 1, 2.357978, 1.943743, 0.682128, 1], ['1.6', '51', 0, 18.0, 19.921874999999996, 0, 0, 2.0, 3.0, 2, 1, 1.0, 1.0, 1.0, 1], ['1.56', '102', 0, 26.0, 41.913214990138066, 0, 0, 3.0, 3.0, 1, 0, 1.0, 0.0, 1.0, 1], ['1.763343', '133.952675', 0, 19.885655, 43.08020830022258, 0, 0, 3.0, 3.0, 1, 1, 2.835622, 1.419473, 0.816986, 1], ['1.805445', '119.484614', 1, 27.931432, 36.655863883391646, 0, 0, 2.911312, 3.0, 1, 1, 2.501808, 0.94676, 0.785701, 1], ['1.67', '64', 0, 19.0, 22.948115744558788, 1, 0, 3.0, 3.0, 1, 1, 2.0, 2.0, 1.0, 1], ['1.76', '78', 1, 21.0, 25.18078512

In [3]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 5.2 MB/s 
     |████████████████████████████████| 210 kB 45.6 MB/s 
     |████████████████████████████████| 81 kB 7.7 MB/s 
     |████████████████████████████████| 78 kB 4.2 MB/s 
     |████████████████████████████████| 147 kB 54.9 MB/s 
     |████████████████████████████████| 50 kB 5.5 MB/s 
     |████████████████████████████████| 112 kB 43.2 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=f82dda9dafbc201dcabbe93be1afada3eb49368cf3a43dd50a8d01167c334733
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.1.0
    Uninstalling importlib-metadata-5.1.0:
      Successfully uninstalled importlib-metadata-5.1.0


In [18]:
import optuna  # pip install optuna
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from optuna.integration import LightGBMPruningCallback

In [77]:
import warnings
warnings.filterwarnings('ignore')

In [79]:
def objective(trial, X, y):
    """
    Objective function to be minimized.
    """
    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "dart",
        "num_class": 7,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.5),
        "max_depth": trial.suggest_int("max_depth", 1, 30),
        "num_leaves": trial.suggest_int("num_leaves", 2, 200),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.1, 1.0),
        "subsample": trial.suggest_float("feature_fraction", 0.1, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100)
    }
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1121218)

    cv_scores = np.empty(10)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = lgb.LGBMClassifier(**param)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="multi_logloss",
            early_stopping_rounds=100,
            callbacks=[
                LightGBMPruningCallback(trial, "multi_logloss")
            ],  # Add a pruning callback
        )
        preds = model.predict_proba(X_test)
        cv_scores[idx] = log_loss(y_test, preds)

    return np.mean(cv_scores)

In [80]:
study = optuna.create_study(direction="minimize", study_name="LGBM Classifier")
func = lambda trial: objective(trial, x, y)
study.optimize(func, n_trials=50)

[I 2022-12-23 13:29:21,434] A new study created in memory with name: LGBM Classifier


[1]	valid_0's multi_logloss: 0.977605
[2]	valid_0's multi_logloss: 0.687013
[3]	valid_0's multi_logloss: 0.536306
[4]	valid_0's multi_logloss: 0.44762
[5]	valid_0's multi_logloss: 0.357284
[6]	valid_0's multi_logloss: 0.282136
[7]	valid_0's multi_logloss: 0.22468
[8]	valid_0's multi_logloss: 0.255193
[9]	valid_0's multi_logloss: 0.207005
[10]	valid_0's multi_logloss: 0.164465
[11]	valid_0's multi_logloss: 0.133896
[12]	valid_0's multi_logloss: 0.143434
[13]	valid_0's multi_logloss: 0.120957
[14]	valid_0's multi_logloss: 0.103616
[15]	valid_0's multi_logloss: 0.0888119
[16]	valid_0's multi_logloss: 0.0786206
[17]	valid_0's multi_logloss: 0.0689143
[18]	valid_0's multi_logloss: 0.0608396
[19]	valid_0's multi_logloss: 0.0550716
[20]	valid_0's multi_logloss: 0.0507472
[21]	valid_0's multi_logloss: 0.0512008
[22]	valid_0's multi_logloss: 0.0475395
[23]	valid_0's multi_logloss: 0.0440622
[24]	valid_0's multi_logloss: 0.0403482
[25]	valid_0's multi_logloss: 0.0393987
[26]	valid_0's multi_logl

[I 2022-12-23 13:29:29,707] Trial 0 finished with value: 0.018458764171835028 and parameters: {'learning_rate': 0.3128325781522467, 'max_depth': 30, 'num_leaves': 193, 'feature_fraction': 0.622798372903273, 'min_child_samples': 86}. Best is trial 0 with value: 0.018458764171835028.


[88]	valid_0's multi_logloss: 0.0268346
[89]	valid_0's multi_logloss: 0.027074
[90]	valid_0's multi_logloss: 0.0268043
[91]	valid_0's multi_logloss: 0.0264389
[92]	valid_0's multi_logloss: 0.0276925
[93]	valid_0's multi_logloss: 0.0281596
[94]	valid_0's multi_logloss: 0.0280279
[95]	valid_0's multi_logloss: 0.0277121
[96]	valid_0's multi_logloss: 0.0277296
[97]	valid_0's multi_logloss: 0.028191
[98]	valid_0's multi_logloss: 0.0285254
[99]	valid_0's multi_logloss: 0.0280496
[100]	valid_0's multi_logloss: 0.0287281
[1]	valid_0's multi_logloss: 1.33406
[2]	valid_0's multi_logloss: 1.21047
[3]	valid_0's multi_logloss: 1.12115
[4]	valid_0's multi_logloss: 1.05158
[5]	valid_0's multi_logloss: 0.967682
[6]	valid_0's multi_logloss: 0.891752
[7]	valid_0's multi_logloss: 0.823667
[8]	valid_0's multi_logloss: 0.854601
[9]	valid_0's multi_logloss: 0.79306
[10]	valid_0's multi_logloss: 0.735141
[11]	valid_0's multi_logloss: 0.67773
[12]	valid_0's multi_logloss: 0.698765
[13]	valid_0's multi_logloss

[I 2022-12-23 13:29:40,786] Trial 1 finished with value: 0.1386376873716934 and parameters: {'learning_rate': 0.07743689128788533, 'max_depth': 8, 'num_leaves': 134, 'feature_fraction': 0.5410448100512901, 'min_child_samples': 42}. Best is trial 0 with value: 0.018458764171835028.


...

[1]	valid_0's multi_logloss: 0.865898
[2]	valid_0's multi_logloss: 0.558895
[3]	valid_0's multi_logloss: 0.419424
[4]	valid_0's multi_logloss: 0.336649
[5]	valid_0's multi_logloss: 0.258841
[6]	valid_0's multi_logloss: 0.194892
[7]	valid_0's multi_logloss: 0.14748
[8]	valid_0's multi_logloss: 0.170221
[9]	valid_0's multi_logloss: 0.133794
[10]	valid_0's multi_logloss: 0.10285
[11]	valid_0's multi_logloss: 0.0830855
[12]	valid_0's multi_logloss: 0.0884281
[13]	valid_0's multi_logloss: 0.0742636
[14]	valid_0's multi_logloss: 0.0650348
[15]	valid_0's multi_logloss: 0.0584678
[16]	valid_0's multi_logloss: 0.0513787
[17]	valid_0's multi_logloss: 0.0451163
[18]	valid_0's multi_logloss: 0.0400208
[19]	valid_0's multi_logloss: 0.0365974
[20]	valid_0's multi_logloss: 0.0350662
[21]	valid_0's multi_logloss: 0.0341384
[22]	valid_0's multi_logloss: 0.0317485
[23]	valid_0's multi_logloss: 0.0302871
[24]	valid_0's multi_logloss: 0.0288718
[25]	valid_0's multi_logloss: 0.0272685
[26]	valid_0's multi_

[I 2022-12-23 13:32:05,956] Trial 47 pruned. Trial was pruned at iteration 25.


[6]	valid_0's multi_logloss: 0.165506
[7]	valid_0's multi_logloss: 0.125213
[8]	valid_0's multi_logloss: 0.145914
[9]	valid_0's multi_logloss: 0.112401
[10]	valid_0's multi_logloss: 0.0885693
[11]	valid_0's multi_logloss: 0.073738
[12]	valid_0's multi_logloss: 0.077693
[13]	valid_0's multi_logloss: 0.067423
[14]	valid_0's multi_logloss: 0.0565311
[15]	valid_0's multi_logloss: 0.0512999
[16]	valid_0's multi_logloss: 0.0463113
[17]	valid_0's multi_logloss: 0.0433688
[18]	valid_0's multi_logloss: 0.0392134
[19]	valid_0's multi_logloss: 0.0361522
[20]	valid_0's multi_logloss: 0.0337158
[21]	valid_0's multi_logloss: 0.0337222
[22]	valid_0's multi_logloss: 0.0343224
[23]	valid_0's multi_logloss: 0.0324227
[24]	valid_0's multi_logloss: 0.0296191
[25]	valid_0's multi_logloss: 0.0311673
[1]	valid_0's multi_logloss: 0.762048
[2]	valid_0's multi_logloss: 0.463871
[3]	valid_0's multi_logloss: 0.339955


[I 2022-12-23 13:32:06,148] Trial 48 pruned. Trial was pruned at iteration 25.
[I 2022-12-23 13:32:06,210] Trial 49 pruned. Trial was pruned at iteration 0.


[4]	valid_0's multi_logloss: 0.264254
[5]	valid_0's multi_logloss: 0.201203
[6]	valid_0's multi_logloss: 0.14591
[7]	valid_0's multi_logloss: 0.107942
[8]	valid_0's multi_logloss: 0.124451
[9]	valid_0's multi_logloss: 0.0978023
[10]	valid_0's multi_logloss: 0.0767568
[11]	valid_0's multi_logloss: 0.0619113
[12]	valid_0's multi_logloss: 0.0643197
[13]	valid_0's multi_logloss: 0.0546176
[14]	valid_0's multi_logloss: 0.0494167
[15]	valid_0's multi_logloss: 0.0471994
[16]	valid_0's multi_logloss: 0.0429252
[17]	valid_0's multi_logloss: 0.0413858
[18]	valid_0's multi_logloss: 0.0366493
[19]	valid_0's multi_logloss: 0.034533
[20]	valid_0's multi_logloss: 0.0345471
[21]	valid_0's multi_logloss: 0.0327507
[22]	valid_0's multi_logloss: 0.0305503
[23]	valid_0's multi_logloss: 0.030821
[24]	valid_0's multi_logloss: 0.0290636
[25]	valid_0's multi_logloss: 0.0306621


In [81]:
print(f"\tBest value (rmse): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

	Best value (rmse): 0.01376
	Best params:
		learning_rate: 0.41938745658520243
		max_depth: 20
		num_leaves: 40
		feature_fraction: 0.46326845773030517
		min_child_samples: 54


In [82]:
from sklearn.model_selection import KFold, cross_val_score
model = lgb.LGBMClassifier(**study.best_params)
skf = StratifiedKFold(n_splits=10)
scores = cross_val_score(model, x, y, cv=skf)

In [83]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [1.         1.         1.         1.         0.99526066 0.99052133
 0.99526066 1.         0.99526066 0.99526066]
Average CV Score:  0.9971563981042653
Number of CV Scores used in Average:  10
